In [ ]:
# From Brad Garton's gen patch
# Max/help/examples/gen/gen~.flute

# Cook's research article:
# @inproceedings{cook1992meta,
#   title={A meta-wind-instrument physical model, and a meta-controller for real-time performance control},
#   author={Cook, Perry R},
#   booktitle={Proceedings of the international computer music conference},
#   volume={273},
#   year={1992},
#   organization={INTERNATIONAL COMPUTER MUSIC ACCOCIATION}
# }

In [ ]:
import numpy as np
from tqdm import tqdm
import time
import soundfile as sf
from scipy.signal import lfilter

In [ ]:
tube_length= 36
embochure = 20
sample_rate = 44100
nbr_de_sec = 0.5
total_samples = int(sample_rate*nbr_de_sec)

In [ ]:
def noise(duration):
    y = (np.random.rand(duration,1)-0.5)*2
    return y

# def normalize(array):
#     normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))
#     normalized_array = (normalized_array-0.5) * 2 # -6 to 6
#     return normalized_array

def generate_envelope(attack_time, decay_time, sustain_level, release_time, sample_rate, total_samples):
    envelope = np.zeros(total_samples)

    attack_samples = int(attack_time * sample_rate)
    decay_samples = int(decay_time * sample_rate)
    release_samples = int(release_time * sample_rate)
    sustain_samples = total_samples - attack_samples - decay_samples - release_samples

    attack = np.linspace(0, 1, attack_samples)
    decay = np.linspace(1, sustain_level, decay_samples)
    sustain = np.full(sustain_samples, sustain_level)
    release = np.linspace(sustain_level, 0, release_samples)

    envelope[:attack_samples] = attack
    envelope[attack_samples:attack_samples+decay_samples] = decay
    envelope[attack_samples+decay_samples:attack_samples+decay_samples+sustain_samples] = sustain
    envelope[-release_samples:] = release

    return envelope

def dc_blocker(signal, cutoff_freq, fs):
    # Calculate filter coefficients　係数を計算
    t = 1.0 / (2.0 * np.pi * cutoff_freq)
    alpha = t / (t + (1.0 / fs))
    b = [1.0 - alpha, -1.0 + alpha]
    a = [1.0, -alpha]

    # Apply filter フィルターを適用
    filtered_signal = lfilter(b, a, signal)

    return filtered_signal



In [ ]:
# Variables 変数

# Sample Rate サンプルレート
sample_rate = 44100

# Noise ノイズ
burst_noise = noise(total_samples)

# Noise Amplitude ノイズの振幅
noiseamp = 0.3

# Envelope エンベロープ
# Time in ms 時間（ミリ秒）
attack_time = 0.015
decay_time = 0.115
sustain_level = 0.0 # Staccato
release_time = 0.135

# Array for output 出力用配列
# Preallocate output array for efficiency 
# 効率のために出力配列を事前に割り当てる
output = np.empty(total_samples)  
delayTube1 = np.empty(total_samples)
delayTube2 = np.empty(total_samples)
delayEmb = np.empty(total_samples)

# Feedback フィードバック
feedback1 = 0
feedback2 = 0

# Impulse Response Filter sample インパルス応答フィルターサンプル
delayIR = 0
delayDCK = 0

# DC Blocker coefficient DCキラー係数
alpha = 0.5

# Measure Calculation Time 計算時間の計測
start = 0
end = 0


In [ ]:
# Generate Envelope エンベロープの生成
envelope = generate_envelope(attack_time, decay_time, sustain_level, release_time, sample_rate, total_samples)

In [30]:
# Start calculation 計算開始
list_of_embochure = [5, 10, 15, 20]

for k in tqdm(range(len(list_of_embochure)), desc='generate staccato files'):
    embochure = list_of_embochure[k]
    for j in range(15, 80):
        tube_length = j
        for i in range(total_samples):
            # Input 入力
            instantSample = burst_noise[i] * noiseamp
            instantSample += 1.

            # Apply amplitude envelope 振幅エンベロープの適用
            instantSample *= envelope[i]

            # Add Feedback フィードバックの足し算
            if i !=0:
                instantSample += feedback1
            
            # Delay 遅延
            delayEmb[i] = instantSample
            if i > embochure:
                instantSample = delayEmb[i-embochure]

            # Nonlinearity
            instantSample = (instantSample**3) - instantSample
            instantSample *= 0.4

            # Add Feedback　フィードバックの足し算
            if i !=0:
                instantSample += feedback2
            instantSample_to_output = instantSample

            # IR Filter 無限インパルス応答フィルター
            instantSample = instantSample * 0.7 + delayIR * 0.3
            delayIR = instantSample

            # Delay 遅延
            delayTube1[i] = instantSample
            if i > tube_length:
                instantSample = delayTube1[i-tube_length]

            # Feedbacks フィードバック
            feedback1 = instantSample * -0.35
            feedback2 = instantSample * 0.9

            # Output 出力
            output[i] = instantSample_to_output

        # Apply DC Blocker DCキラーを適用
        output = dc_blocker(output, 50, sample_rate)

        # Smooth the 100 first samples 最初の100サンプルを滑らかにする
        output[:100] = np.linspace(0, output[100], 100)

        # Boost Signal of 30 dB, 30dBの信号ブースト
        output *= 31.622776601683793

        # Write output to file 出力をファイルに書き込む
        sf.write('staccato/Fl-stacc-'+str(embochure)+str(tube_length)+'.wav', output, sample_rate)

generate staccato files: 100%|██████████| 4/4 [01:12<00:00, 18.23s/it]
